In [1]:
# This process is now automated for Elementary school Lunch in English and Spanish.
# Just do Runtime > Run all (Ctrl+F9)
# Then download the file and import it into Google Calendar, etc. Open the file menu on the left with the folder icon

# If the fragile formatting code breaks, I may need to update things.
# Also perhaps for a new calendar month format or for a menu type

In [2]:
# Install packages and libraries
!pip install pypdf ics
!pip install emoji nltk
!pip install beautifulsoup4 requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 22.2 MB/s eta 0:00:00


In [3]:
# prompt: use pydpf to read text from a pdf at a given url, parse the entries and create an ics-formatted text string for a calendar program import
from urllib.request import urlopen
from urllib.parse import urlparse, urlunparse, quote
from io import BytesIO
from pypdf import PdfReader
from ics import Calendar, Event
import ics
from datetime import datetime
import re
import pytz
import emoji
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import requests
from bs4 import BeautifulSoup

# Create WordNetLemmatizer object
wnl = WordNetLemmatizer()


def get_all_links(url):
    try:
        # Send an HTTP GET request to the URL
        response = requests.get(url)
        # Check if the request was successful
        response.raise_for_status()
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        # Find all anchor tags and extract the href attribute
        links = [a.get('href') for a in soup.find_all('a', href=True)]
        return links
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the URL: {e}")
        return []


def url_to_text(pdf_url):
    try:
        remote_file = urlopen(pdf_url).read()
        memory_file = BytesIO(remote_file)
        reader = PdfReader(memory_file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    return text


def is_valid_date(date_string, language):
    """
    Checks if the input string is a valid date in the format "Month Day"
    where the month is a word (e.g., January, February) and the day is a number.
    """
    if language == 'es':
        pattern = r"^(Enero|Febrero|Marzo|Abril|Mayo|Junio|Julio|Agosto|Septiembre|Octubre|Noviembre|Diciembre)\s+(\d{1,2})$"
    else:
        pattern = r"^(January|February|March|April|May|June|July|August|September|October|November|December)\s+(\d{1,2})$"

    match = re.match(pattern, date_string)
    if not match:
        return False

    month_name = match.group(1)
    day = int(match.group(2))

    if language == 'es':
        return True
    else:
        try:
            datetime.strptime(f"{month_name} {day}", "%B %d")
            return True
        except ValueError:
            return False


def parse_date_string(date_string, language):
    """Parses a date string into month, day, and year.

    Args:
      date_string: The date string to parse.  Must be in the format "Month Day".

    Returns:
      A tuple containing the month (string), day (int), and year (int).
      Returns None if the date string is invalid.
    """
    if language == 'es':
        match = re.match(r"^(Enero|Febrero|Marzo|Abril|Mayo|Junio|Julio|Agosto|Septiembre|Octubre|Noviembre|Diciembre)\s+(\d{1,2})$", date_string)
    else:
        match = re.match(r"^(January|February|March|April|May|June|July|August|September|October|November|December)\s+(\d{1,2})$", date_string)
    if not match:
        return None
    month_name = match.group(1)
    day = int(match.group(2))

    if language == 'es':
        # list of month names
        month_names = ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]
        month = month_names.index(month_name)+1
        year = datetime.now().year
        return month, day, year
    elif language == 'en':
        try:
            date_obj = datetime.strptime(f"{month_name} {day}", "%B %d")
            month = date_obj.strftime("%B")  # Get month name
            day = date_obj.day
            year = datetime.now().year
            return month, day, year
        except ValueError:
            return None
    else:
        return None


def text_to_ics(text, event_title, language):
    calendar = Calendar()
    lines = text.split('\n')
    #print(lines)
    i=0
    while i < len(lines):
        line = lines[i]
        if is_valid_date(line.strip(), language):
            date_parts = parse_date_string(line.strip(), language)
            if date_parts is not None:
                month, day, year = date_parts
                event = Event()
                event.name = event_title
                if language == 'en':
                    event.begin = datetime(year, datetime.strptime(month, "%B").month, day, tzinfo=pytz.utc) # set begin time
                else:
                    event.begin = datetime(year, month, day, tzinfo=pytz.utc) # set begin time
                event.make_all_day()
                extra_content = ics.utils.ContentLine(name="TRANSP", value="TRANSPARENT")
                event.extra.append(extra_content)
                # build description until you reach next date
                event.description = ''
                while i + 1 < len(lines) and not is_valid_date(lines[i + 1].strip(), language) and "Prices" not in lines[i + 1] and "Precios" not in lines[i + 1]:
                    next_line = lines[i + 1].strip()
                    nocolons_next_line: str = re.sub(r':', r'', next_line)
                    low = nocolons_next_line.lower()
                    lemmatized_string = ' '.join([wnl.lemmatize(words) for words in low.split()])
                    colons: str = re.sub(r'(\w*)', r':\1:', lemmatized_string)
                    line = nocolons_next_line + colons
                    #print(line)
                    with_emojis: str = emoji.emojize(string=line, language=language)
                    nocolons: str = re.sub(r':(\w*):', r'', with_emojis)
                    event.description += nocolons + '\n'
                    i += 1
                calendar.events.add(event)
        else:
            i += 1
    return calendar


def to_file(ics_string, filename):
    if ics_string:
        # To save to file:
        with open(filename, "w", newline='', encoding='utf-8') as f:
            f.write(ics_string)

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [4]:
# Now we find the menu from the site
url = 'https://www.dpsnc.net/Page/7089'
baseUrl = 'https://www.dpsnc.net'
all_links = get_all_links(url)
pdf_files = [file for file in all_links if file.endswith('.pdf')]

In [7]:
def wrap(baseUrl, pdf_files, language):
    if 'en' in language:
        relative_url = [file for file in pdf_files if 'ES' in file and 'Lunch' in file and not 'Spanish' in file]
        event_title = 'DPS Lunch Menu'  # All events in the calendar will have this title
        outfile = 'english.ics'
    elif 'es' in language:
        relative_url = [file for file in pdf_files if 'ES' in file and 'Lunch' in file and 'Spanish' in file]
        event_title = 'DPS Almuerzo Menu'  # All events in the calendar will have this title
        outfile = 'espanol.ics'
    else:
        return False

    link = baseUrl + relative_url[0][5:].replace(" ", "%20")
    #print(link)
    text = url_to_text(link)
    #print(text)
    cal = text_to_ics(text, event_title, language)
    ics_string = cal.serialize()
    #print(ics_string)
    to_file(ics_string, outfile)

    return True

In [8]:
# English code here
language = 'en'
wrap(baseUrl, pdf_files, language)

True

In [9]:
# Spanish code here
language = 'es'
wrap(baseUrl, pdf_files, language)

True